In [1]:
import torch
from transformers import AutoTokenizer, BertForMaskedLM
from utils.saver import tokenizer_loader, model_loader

tokenizer = tokenizer_loader(AutoTokenizer, "bert-base-chinese")
model = model_loader(BertForMaskedLM, "bert-base-chinese")

In [2]:
from operators.PTEDataset import PTEDataset

DATASET_NAME = "msra.entail.min"
train_dataset = PTEDataset(tokenizer=tokenizer, reader=f"./prompts/{DATASET_NAME}.train.tsv")
dev_dataset = PTEDataset(tokenizer=tokenizer, reader=f"./prompts/{DATASET_NAME}.dev.tsv")
train_dataset.flag

['对', '错']

In [3]:
LEARNING_RATE = 5e-3
EPOCH = 1
BATCH_SIZE = 4

def train_loop(train_dataset, dev_dataset, model):
    optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
    for _ in range(EPOCH):
        train(train_dataset, model, optimizer)
        # TODO: finish test()

def train(dataset, model, optimizer):
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE)
    for batch_index, (batch_X, batch_Y) in enumerate(dataloader):
        loss = model(**batch_X, labels=batch_Y).loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [4]:
train_loop(train_dataset, dev_dataset, model)
model.save_pretrained("./pretrained/model/fine-tune/prompt-entail")